# Programming Project - Unit 3,1
*by Igor A. Brandão and Leandro Antonio Feliciano da Silva*

**Goals**
The purpose of this project is explore the following:

- Access a database content;
- Full content of the statistical part seen in the course;
- Graphs generation;
- Geolocation analysis and hypotheses should be explained in detail.

## Database connection

In the cell bellow, we perform a connection with local database.

**Important:** The database file must be in the same notebook's folder or it should be correctly referenced

In [1]:
# Import necessary module
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
engine = create_engine('sqlite:///py-students-database.sqlite')

In [2]:
# What are the tables in the database?

# Save the table names to a list: table_names
table_names  = engine.table_names()

# Print the table names to the shell
print(table_names)

['Grade']


## Database query

Here we perform a basic query into DB table to check if it's correctly connected

In [5]:
# Query string
sql = "select * from Grade"

# Execute query and store records in DataFrame: df
df = pd.read_sql_query(sql, engine)

# drop the unnecessary column
df = df.drop('index', 1)

# print the number of unique students
print(len(df['Aluno_ID'].unique()))

# Print head of DataFrame
df.head()

766


,Aluno_ID,Período,Disciplina,Situaçăo,Média Final,Unidade I,Unidade II,Unidade III
0,0,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.0,4.9,9.0,7.0
1,1,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.3,8.0,7.0,7.0
2,2,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,9.3,9.5,8.3,10.0
3,3,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.3,6.5,7.0,8.3
4,4,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.5,5.5,8.0,9.0


## Data export [optional]

In order to visualize the data into an excel file, the cell bellow is responsible for exporting the data.

In [6]:
# Export the new dataSet to csv
df.to_csv('py-students-dataset.csv', encoding="utf-8")